# Исследование платежеспособности клиентов банка

Влияет ли семейное положение клиентов и количество детей на факт погашения кредита в срок?

Чтобы ноутбук работал локально, требуется установка библиотеки для лемматизации (ячейка ниже):

In [1]:
!pip install pymystem3 -U

Defaulting to user installation because normal site-packages is not writeable


## Импорт библиотек и предобработка данных

In [2]:
import pymystem3

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from pymystem3 import Mystem
import warnings

warnings.filterwarnings("ignore")

In [4]:
df=pd.read_csv('borrorers_data.csv')
df.sample(n=5, random_state=1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1383,0,353802.811675,37,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452.226085,строительство недвижимости
300,1,-359.193975,33,СРЕДНЕЕ,1,гражданский брак,1,M,сотрудник,0,223001.623994,на проведение свадьбы
6565,2,-1064.854333,35,среднее,1,гражданский брак,1,F,компаньон,0,163591.209323,свадьба
17027,0,NaN,48,высшее,0,гражданский брак,1,F,сотрудник,0,NaN,операции с жильем
4077,0,-7059.100220,45,высшее,0,гражданский брак,1,F,компаньон,1,194820.185757,сыграть свадьбу


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


Как видно из описания, в таблице есть 21525 записи, в 2 столбцах допущены пропуски days_employed и total_income, по 2174 записи.

## <span style='color:green'>Предобработка данных </span>

Посмотрим уникальные значения каждого из столбцов, например у детей обнаружились записи -1 и 20 детей.

In [7]:
sorted(df['children'].unique())

[-1, 0, 1, 2, 3, 4, 5, 20]

47 записей с количеством детей -1 и 76 записей для клиентов с 20 детьми (!!!)

In [8]:
df[df['children'] == -1]['children'].count()

47

In [9]:
df[df['children'] == 20]['children'].count()

76

Если посмотреть среднее и медиану по столбцу children, исключая значение -1, то получится 0.0 и 0.54
если посмотреть среднее и медиану по столбцу children, исключая значение 20, то получится 0.0 и 0.47

In [10]:
print(df[df['children'] != 20]['children'].median())
print(df[df['children'] != 20]['children'].mean())


0.0
0.4699519791132454


Принимаю решение заменить наблюдения -1 и 20 нулями. На всякий случай поискала статистику, в 2018 году на 23,75 млн семей в России приходилось около 943 семьи, воспитывающей 11 детей и более, частота получается намного меньше,чем если смотреть в нашей таблице. Также, если смотреть на все уникальные наблюдений, в ряду после 5 детей нет наблюдений с 6 до 19 (а это невозможно, такой резкий разброс), значит значение 20 детей также ошибочно.

In [11]:
df.loc[df['children'] == -1, 'children'] = 0
df.loc[df['children'] == 20, 'children'] = 0

df['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

В days_employed есть немножко чертовщины, положительные, отрицательные, дробные. 

In [12]:
df['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

сначала избавимся от минуса в стаже, скорее всего случайно поставили. Например, иногда минус можно спутать с тире н лет - к дней

In [13]:
df['days_employed'] = df['days_employed'].abs()

оценим метрики столбцов days_employed

In [14]:
print(df['dob_years'].agg(['min', 'max','mean', 'median']))
print(df['days_employed'].agg(['min', 'max','mean', 'median']))

min        0.00000
max       75.00000
mean      43.29338
median    42.00000
Name: dob_years, dtype: float64
min           24.141633
max       401755.400475
mean       66914.728907
median      2194.220567
Name: days_employed, dtype: float64


In [15]:
df['days_employed'].isna().sum()

2174

2174 записи составляют 10,09% от всех записей таблицы сейчас.

In [16]:
def fillmedian(age):
    ''' функция считает медиану трудового стажа в днях для фиксированного возраста в таблице'''
    return df[df['dob_years'] == age]['days_employed'].median()

In [17]:
nan_indexes = df.loc[df['days_employed'].isna()].index.tolist()
#формируем список индексов строк с пропущенными значениями в стаже
for item in nan_indexes:
# заменяем каждое пропущенное медианой по возрасту, указанному в этой же строке    
    df.loc[item,'days_employed'] = fillmedian(df.loc[item,'dob_years']) 
df['days_employed'].isna().sum()

0

In [18]:
df.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

исправим тип данных в столбце с вещественного на целый (количество лет клиента целое, количество дней стажа тоже должно быть целым)

In [19]:
df['days_employed'] = df['days_employed'].astype('int')
display(df['days_employed'].unique())
display(df['days_employed'].agg(['min', 'max','mean', 'median']))

array([  8437,   4024,   5623, ..., 362161, 373995, 343937])

min           24.000000
max       401755.000000
mean       66931.591266
median      2170.000000
Name: days_employed, dtype: float64

In [20]:
for dtype in ['float','int','object']:
    selected_dtype = df.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 0.08 MB
Average memory usage for int columns: 0.13 MB
Average memory usage for object columns: 1.69 MB


In [21]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # исходим из предположения о том, что если это не DataFrame, то это Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(df_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([df_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

0.90 MB
0.18 MB


,before,after
uint8,NaN,5.0
int32,1.0,NaN
uint32,NaN,1.0
int64,5.0,NaN


<span style='color:blue'> с помощью вот такого преобразования типов снизилось потребление памяти на 82% </span> 

In [22]:
df_float = df.select_dtypes(include=['float'])
converted_float = df_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(df_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([df_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)


0.16 MB
0.08 MB


,before,after
float32,NaN,1.0
float64,1.0,NaN


<span style='color:blue'> объем памяти, необходимый для хранения типов с плавающей точкой  сократился на 50% </span> 

In [23]:
optimized_df = df.copy()

optimized_df[converted_int.columns] = converted_int
optimized_df[converted_float.columns] = converted_float

print(mem_usage(df))
print(mem_usage(optimized_df))

11.20 MB
10.40 MB


<span style='color:blue'> потребление памяти сократилось на 5%</span>  

при сортировке уникальных значений возрастов клиентов обнаружился клиент возрастом 0 лет (???)
print использован, чтобы вывести уникальные значения в строчку

In [24]:
print(sorted(df['dob_years'].unique()))

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


In [25]:
df['dob_years'].agg(['min', 'max','mean', 'median'])

min        0.00000
max       75.00000
mean      43.29338
median    42.00000
Name: dob_years, dtype: float64

медианный возраст клиентов, без учета нулей 42 года, средний возраст 43, принимаю решение заменить значение 0 на значение 42

In [26]:
df[df['dob_years'] != 0]['dob_years'].mean()

43.497479462285284

In [27]:
df['dob_years']= df['dob_years'].replace(0,42)

In [28]:
df[df['dob_years']==0]['dob_years'].count()

0

при просмотре уровней образования явно возникла необходимость привести все к одному регистру

In [29]:
df['education'] = df['education'].str.lower()

In [30]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

все правильно, уникальные значения 5 штук для идентификатора образования

In [31]:
df['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

Небольшая ремарка по поводу education_id и education. Можно было бы вынести в отдельную таблицу столбец education, оставив в этой только education_id, это бы сократило объем необходимой памяти для хранения,и немного упростило таблицу, тк education_id является категориальной переменной (но неправильно сформированной, к сожалению). Но, как видно из ячейки ниже, значения education_id не упорядочены. И если бы потребовалось ответить на вопрос, есть ли зависимость между образованием клиента и риском невозврата кредита, то тогда бы да, надо было аккуратнее использовать эти переменные. Но такая задача не ставилась в рамках проекта. 

Я не могу сказать, что это ошибка - то, как формировались education_id или education Со своей стороны, но для использования в анализе, я бы переназначила значения и уровни образования:
0 - начальное
1 - среднее
2 - неоконченное высшее
3 -  высшее
4-  ученая степень

In [32]:
for item in ['высшее', 'среднее', 'неоконченное высшее', 'начальное', 'ученая степень']:
    print(item, df[df['education'] == item]['education_id'].value_counts())

высшее 0    5260
Name: education_id, dtype: int64
среднее 1    15233
Name: education_id, dtype: int64
неоконченное высшее 2    744
Name: education_id, dtype: int64
начальное 3    282
Name: education_id, dtype: int64
ученая степень 4    6
Name: education_id, dtype: int64


In [33]:
df['education_id'] = df['education_id'].replace(3,5)   
df['education_id']= df['education_id'].replace(0,3)  
df['education_id']= df['education_id'].replace(5,0)  
for item in ['начальное','среднее','неоконченное высшее', 'высшее', 'ученая степень']:
    print(item, df[df['education'] == item]['education_id'].value_counts())

начальное 0    282
Name: education_id, dtype: int64
среднее 1    15233
Name: education_id, dtype: int64
неоконченное высшее 2    744
Name: education_id, dtype: int64
высшее 3    5260
Name: education_id, dtype: int64
ученая степень 4    6
Name: education_id, dtype: int64


у семейного статуса и идентификатора семейного статуса "странных" значений не обнаружено

In [34]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [35]:
df['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

Если сопоставить категории семейного положения и семейный статус, то я не могу сказать, что не женат "лучше", чем "женат", строго говоря их нельзя упорядочить. Да, family_status_id однозначно определяет family_status, но в решении задачи дальше я использовала только family_status, для наглядности.

In [36]:
for item in ['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем']:
    print(item, df[df['family_status'] == item]['family_status_id'].value_counts())

женат / замужем 0    12380
Name: family_status_id, dtype: int64
гражданский брак 1    4177
Name: family_status_id, dtype: int64
вдовец / вдова 2    960
Name: family_status_id, dtype: int64
в разводе 3    1195
Name: family_status_id, dtype: int64
Не женат / не замужем 4    2813
Name: family_status_id, dtype: int64


В столбце ежемесячный доход сделаем оценку метрик, и количества пропущенных значений. Заменим пропущенные на 

In [37]:
df['total_income'].agg(['min', 'max','mean', 'median'])

min       2.066726e+04
max       2.265604e+06
mean      1.674223e+05
median    1.450179e+05
Name: total_income, dtype: float64

In [38]:
df['total_income'].isna().sum()

2174

In [39]:
medium_income = df['total_income'].median()
df['total_income']= df['total_income'].fillna(medium_income)
df['total_income'].isna().sum()

0

В столбце пол возникла проблема -у нас есть люди с неопределенным (или неопределившимся полом), что значит XNA ???

In [40]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Попробуем оценить средний доход для людей каждого пола.
Согласно записи РБК от 29 марта 2019 (сейчас не буду искать), по подсчетам Института экономики РАН, 
в среднем в России женщины получают сейчас на 27% меньше мужчин. В Москве разница намного
меньше — всего 11%. Если разница в доходе в нашей таблице будет различаться, попробуем заменить 
наблюдение 'XNA'на тот пол , к среднему доходу которого значение будет ближе
и еще доклад фбк: 

https://www.fbk.ru/upload/medialibrary/bce/Gendernyi%20razryv%20v%20oplate%20truda_doklad.pdf
В последние годы в России отмечается сокращение гендерного разрыва в
оплате труда работников в пользу мужчин: с 39,3% в 2005 году до 27,4% в
2015 году. В странах Европейского союза наблюдается аналогичная
тенденция, хотя и гораздо менее выраженная: гендерный разрыв сократился с
17,7% в 2006 году до 16,4% в 2015 году.

In [41]:
df.groupby('gender')['total_income'].mean()

gender
F      153151.047940
M      188610.831861
XNA    203905.157261
Name: total_income, dtype: float64

как видим из средних значений, средний доход женщин порядка 154 тысячи, а мужчин 193 тысячи. Загадочные XNA имеют средний доход даже больше мужчин, 203 тысячи. поэтому заменим XNA на M

In [42]:
df['gender']= df['gender'].replace('XNA','M')
df['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

в столбце тип занятости тоже все в порядке

In [43]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

столбец имел ли задолженность логичен - либо имел, либо не имел

In [44]:
df['debt'].unique()

array([0, 1], dtype=int64)

Причин кредитов много. Применим лемматизацию, классифицируем причины, выделим категории, и тогда все станет понятнее

In [45]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [46]:
df_purpose_unique = list(df['purpose'].unique())
m = Mystem()
lemmas_list=[]
for row in df_purpose_unique:
    lemmas = m.lemmatize(row)
    for item in lemmas:
        if item not in lemmas_list:
            lemmas_list.append(item)
print(lemmas_list)

['покупка', ' ', 'жилье', '\n', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'с', 'на', 'проведение', 'для', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'со', 'заниматься', 'сделка', 'подержанный', 'получение', 'высокий', 'профильный', 'сдача', 'ремонт']


немножко почистим наш получившийся список. Потом подумала, и решила строительство тоже убрать из уникальных категорий, (строительство отнесла к недвижимости)

In [47]:
unnecessary_values = ['покупка', ' ', '\n', 'приобретение', 'дополнительный', 'жилье',
                     'сыграть', 'операция', 'с', 'на', 'проведение', 'для', 'семья',
                     'коммерческий', 'жилой', 'собственный', 'подержать', 'свой', 'со',
                      'заниматься', 'сделка', 'подержанный', 'получение', 'высокий', 'профильный', 'сдача']
for value in unnecessary_values:
    lemmas_list.remove(value)

print(lemmas_list)

['автомобиль', 'образование', 'свадьба', 'недвижимость', 'строительство', 'ремонт']


In [48]:
def purpose_group(purpose): 
    '''Функция ищет по ключевому слову в фразе с целью совпадения и присваивает строке в таблице категорию '''
    purpose_words = purpose.split()
    for word in purpose_words:
        if 'автомобил' in word:
            return 'автомобиль'
        if 'образован' in word:
            return 'образование'
        if 'свадьб' in word:
            return 'свадьба'
        if ('недвижимост' in word) or ('жил' in word):
            return 'недвижимость'
        if 'ремонт' in word:
            return 'ремонт'

In [49]:
df['purpose_new'] = df['purpose'].apply(purpose_group) 
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new
0,1,8437,42,высшее,3,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба


проверим, заполнился ли новый столбец полностью, нет ли там пропусков (пропусков нет)

In [50]:
df['purpose_new'].isna().sum()

0

удалю старый столбец с целями

In [51]:
df.drop('purpose', axis=1, inplace=True)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_new
0,1,8437,42,высшее,3,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба


Вот теперь посмотрим дубликаты в таблице. Так как метод .duplicated() ищет именно полные дубликаты строчки, вполне можем их удалить. 389 дубликатов это 1,8% от первоначального объема таблицы.

In [52]:
len(df) - len(df['total_income'].unique())

2174

Какова вероятность полного совпадения значения дохода в столбце total_income? В силу большой вариативности этого столбца, и если посмотреть, отнять от общей длины датафрейма длину массива из уникальных значений total_income, то мы получим как раз количество пропусков в столбце total_income, а значит все суммы были уникальными. И в некотором смысле, идентифицируют строки уникально (хотя я не могу сказать, что это хорошо - идентифицировать строки по числовому параметру)

In [53]:
df.duplicated().sum()

386

еще раз проверяем, остались ли дубликаты - нет. Удаляем с обновлением индекса строк в таблице.

In [54]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

Какие могут быть причины появления дубликатов? Например, повторные заявки клиентов на кредит. Когда они писали тоже самое, но другими словами в текстовой причине кредита. 
И еще раз взглянем на свойства таблицы - теперь все хорошо.

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21139 entries, 0 to 21138
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21139 non-null  int64  
 1   days_employed     21139 non-null  int32  
 2   dob_years         21139 non-null  int64  
 3   education         21139 non-null  object 
 4   education_id      21139 non-null  int64  
 5   family_status     21139 non-null  object 
 6   family_status_id  21139 non-null  int64  
 7   gender            21139 non-null  object 
 8   income_type       21139 non-null  object 
 9   debt              21139 non-null  int64  
 10  total_income      21139 non-null  float64
 11  purpose_new       21139 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


##  Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для начала посмотрим, что 0 - не имел задолженности (19397 наблюдений), 1 - имел задолженность (1739 наблюдений)

In [56]:
df['debt'].value_counts()

0    19400
1     1739
Name: debt, dtype: int64

Я использовала свои наработки из другого проекта. В целом, пока решала данную задачу, общее число заявок сделала и обычной группировкой. Но никак не могла разобраться, как посчитать доли заявок (что на мой взгляд информативнее, чем абсолютные значения)

In [57]:
def number_children(children, debt): 
    '''функция, которая сначала создает отфильтрованный
    по параметрам children (количество детей), debt (были ли просрочки) DataFrame, 
    а потом подсчитывает количество строк, удовлетворяющих фильтру '''    
    children_list = df[df['children'] == children]  
    children_list = children_list[children_list['debt'] == debt] 
    children_list_count = children_list['children'].count()  
    return children_list_count

сделаем сводную таблицу, рассчитывающую доли заявок с просрочкой в зависимости от количества детей

In [58]:
name_columns = ['child_count','0', '1', '2', '3', '4', '5']
request_counts = []
for item_debt in [0, 1]:
    row = []
    row.append(item_debt)
    for item_child in [0, 1, 2, 3, 4, 5]:
        row.append(number_children(item_child, item_debt))
    request_counts.append(row)
request_child = pd.DataFrame(data = request_counts ,columns = name_columns, index = ['нет просрочки', 'есть просрочка'])
request_child['>2'] = request_child['3'] + request_child['4'] + request_child['5']
request_child.drop(['3','4','5'], axis=1, inplace=True)
for item in ['0','1','2','>2']:
    request_child[item] = request_child[item] / request_child[item].sum()*100

request_child

,child_count,0,1,2,>2
нет просрочки,0,92.337988,90.666386,90.485532,91.798942
есть просрочка,1,7.662012,9.333614,9.514468,8.201058


Нас интересует в первую очередь вторая строка, и в целом можно утверждать, что с увеличением детей доля просроченных кредитов будет увеличиваться. Данных по просрочке с  5 детьми нет в таблице.
Самая большая доля просрочек возникает в случае 4 детей. Самая низкая частота просрочек в случае 0 детей.

In [59]:
df['children'].value_counts()

0    13965
1     4757
2     2039
3      329
4       40
5        9
Name: children, dtype: int64

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [60]:
df['family_status'].value_counts()

женат / замужем          12093
гражданский брак          4124
Не женат / не замужем     2785
в разводе                 1193
вдовец / вдова             944
Name: family_status, dtype: int64

In [61]:
def number_family(status, debt): 
    '''функция, которая сначала создает отфильтрованный
    по параметрe status (семейное положение), debt (были ли просрочки) DataFrame, 
    а потом подсчитывает количество строк, удовлетворяющих фильтру '''    
    family_list = df[df['family_status'] == status]  
    family_list = family_list[family_list['debt'] == debt] 
    family_list_count = family_list['family_status'].count()  
    return family_list_count

In [62]:
name_columns = ['family_count','женат / замужем', 'гражданский брак', 'Не женат / не замужем', 'в разводе', 'вдовец / вдова']
request_counts = []
for item_debt in [0, 1]:
    row = []
    row.append(item_debt)
    for item_family in ['женат / замужем', 'гражданский брак', 'Не женат / не замужем', 'в разводе', 'вдовец / вдова']:
        row.append(number_family(item_family, item_debt))
    request_counts.append(row)
request_family = pd.DataFrame(data = request_counts ,columns = name_columns, index = ['нет просрочки', 'есть просрочка'])
for item in ['женат / замужем', 'гражданский брак', 'Не женат / не замужем', 'в разводе', 'вдовец / вдова']:
    request_family[item] = request_family[item] / request_family[item].sum()*100
request_family

,family_count,женат / замужем,гражданский брак,Не женат / не замужем,в разводе,вдовец / вдова
нет просрочки,0,92.31787,90.591659,90.16158,92.875105,93.326271
есть просрочка,1,7.68213,9.408341,9.83842,7.124895,6.673729


Самая большая доля просрочек по кредитам возникает в случае неженатых/незамужних клиентов. Самая низкая доля просрочки в случае семейного положения вдовец / вдова. В целом можно сказать, что семейный статус влияет на частоту возникновения просрочек по кредитам.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

выделить категории доходов, сделать доп столбец, написать функцию см практикум

In [63]:
df['total_income'].agg(['min', 'max','mean', 'median'])

min       2.066726e+04
max       2.265604e+06
mean      1.655273e+05
median    1.450179e+05
Name: total_income, dtype: float64

Для решения задач сделаем новую категориальную переменную, разобьем диапазон значений доходов на 5 равных интервалов, напишем функцию

In [64]:
def income_rank(amount):
    ''' функция присваивает категорию от очень низкого до очень высокого сумме дохода, весь диапазон разбит на 5 категорий'''
    d = (df['total_income'].max() - df['total_income'].min() ) / 5
    if (amount >= df['total_income'].min()) and (amount < df['total_income'].min() + d):
        return 'очень низкий'
    if (amount >= df['total_income'].min() + d) and (amount < df['total_income'].min() + 2*d):
        return 'низкий'
    if (amount >= df['total_income'].min()+2*d) and (amount < df['total_income'].min() + 3*d):
        return 'средний'
    if (amount >= df['total_income'].min() + 3*d) and (amount < df['total_income'].min() + 4*d):
        return 'высокий'
    if (amount >= df['total_income'].min() + 4*d) and (amount < df['total_income'].min() + 5*d):
        return 'очень высокий'

In [65]:
df['income_status'] = df['total_income'].apply(income_rank) 
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_new,income_status
0,1,8437,42,высшее,3,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость,очень низкий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль,очень низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость,очень низкий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование,очень низкий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба,очень низкий


In [66]:
def number_income(income, debt): 
    '''функция, которая сначала создает отфильтрованный
    по параметрe income (уровень дохода), debt (были ли просрочки) DataFrame, 
    а потом подсчитывает количество строк, удовлетворяющих фильтру '''    
    income_list = df[df['income_status'] == income]  
    income_list = income_list[income_list['debt'] == debt] 
    income_list_count = income_list['income_status'].count()  
    return income_list_count

In [67]:
name_columns = ['purpose_count','очень низкий', 'низкий', 'средний', 'высокий', 'очень высокий']
request_counts = []
for item_debt in [0, 1]:
    row = []
    row.append(item_debt)
    for item_income in ['очень низкий', 'низкий', 'средний', 'высокий', 'очень высокий']:
        row.append(number_income(item_income, item_debt))
    request_counts.append(row)
request_income = pd.DataFrame(data = request_counts ,columns = name_columns, index = ['нет просрочки', 'есть просрочка'])
for item in ['очень низкий', 'низкий', 'средний', 'высокий', 'очень высокий']:
    request_income[item] = request_income[item] / request_income[item].sum()*100
request_income

,purpose_count,очень низкий,низкий,средний,высокий,очень высокий
нет просрочки,0,91.7374,94.552529,95.238095,100.0,0.0
есть просрочка,1,8.2626,5.447471,4.761905,0.0,100.0


Лично меня смутило то, что сначала наблюдалась четкая зависимость между увеличением дохода и уменьшением риска просрочки (уменьшением доли просрочки), но затем в категориях высокий и очень высокий доход что-то произошло. Посморим, сколько вообще записей попадает в эти категории дохода:

In [68]:
df['income_status'].value_counts()

очень низкий     20853
низкий             257
средний             21
высокий              6
очень высокий        1
Name: income_status, dtype: int64

Теперь я конечно понимаю, что разбиение на категории доходов было крайне неудачным. Напишем еще одну функцию, которая сделает разделение на категории по другим интервалам

In [69]:
right_border = df['total_income'].min() + (df['total_income'].max()-df['total_income'].min())*2/5
right_border

918641.9697650606

In [70]:
right_border = df['total_income'].min() + (df['total_income'].max()-df['total_income'].min())*2/5
d = (right_border - df['total_income'].min() ) / 5
df['total_income'].min() + d

200262.20498762937

In [71]:
def income_rank_bordered(amount):
    ''' функция присваивает категорию от очень низкого до очень высокого сумме дохода,
    но берет не весь размах дохода, а только большую часть, весь диапазон разбит на 5 категорий'''
    right_border = df['total_income'].min() + (df['total_income'].max()-df['total_income'].min())*2/5
    d = (right_border - df['total_income'].min() ) / 5
    if (amount >= df['total_income'].min()) and (amount < df['total_income'].min() + d):
        return 'очень низкий'
    if (amount >= df['total_income'].min() + d) and (amount < df['total_income'].min() + 2*d):
        return 'низкий'
    if (amount >= df['total_income'].min()+2*d) and (amount < df['total_income'].min() + 3*d):
        return 'средний'
    if (amount >= df['total_income'].min() + 3*d) and (amount < df['total_income'].min() + 4*d):
        return 'высокий'
    if (amount >= df['total_income'].min() + 4*d) and (amount < df['total_income'].min() + 5*d):
        return 'очень высокий'

In [72]:
df['income_status_bordered'] = df['total_income'].apply(income_rank_bordered) 
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_new,income_status,income_status_bordered
0,1,8437,42,высшее,3,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость,очень низкий,низкий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль,очень низкий,очень низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость,очень низкий,очень низкий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование,очень низкий,низкий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба,очень низкий,очень низкий


In [73]:
def number_income_bordered(income, debt): 
    '''функция, которая сначала создает отфильтрованный
    по параметрe income (уровень дохода), debt (были ли просрочки) DataFrame, 
    а потом подсчитывает количество строк, удовлетворяющих фильтру '''    
    income_list = df[df['income_status_bordered'] == income]  
    income_list = income_list[income_list['debt'] == debt] 
    income_list_count = income_list['income_status_bordered'].count()  
    return income_list_count

In [74]:
name_columns = ['purpose_count','очень низкий', 'низкий', 'средний', 'высокий', 'очень высокий']
request_counts = []
for item_debt in [0, 1]:
    row = []
    row.append(item_debt)
    for item_income in ['очень низкий', 'низкий', 'средний', 'высокий', 'очень высокий']:
        row.append(number_income_bordered(item_income, item_debt))
    request_counts.append(row)
request_income = pd.DataFrame(data = request_counts ,columns = name_columns, index = ['нет просрочки', 'есть просрочка'])
for item in ['очень низкий', 'низкий', 'средний', 'высокий', 'очень высокий']:
    request_income[item] = request_income[item] / request_income[item].sum()*100
request_income

,purpose_count,очень низкий,низкий,средний,высокий,очень высокий
нет просрочки,0,91.406201,92.761516,94.176707,95.652174,90.0
есть просрочка,1,8.593799,7.238484,5.823293,4.347826,10.0


Все еще есть проблема в столбце очень высокие доходы. Но тогда надо было выяснять при получении данных, какие значения дохода считать "неправильными", и договариваться с заказчиком, что делать с этими цифрами - удалять данные строки, или делать их средними. Но, несмотря на эту деталь, прослеживается четкая зависимость, чем выше доход, чем меньше шанс возникновения просрочки платежа, максимальный риск просрочки 8,5 % у категории с очень низким доходом, от 20667 р до 200262 р.
Либо пересчитать еще раз границы категориального разделения так, чтобы в каждой категории находилось примерно одинаковое количество строк.

In [75]:
df['income_status_bordered'].value_counts()

очень низкий     16093
низкий            4407
средний            498
высокий             92
очень высокий       20
Name: income_status_bordered, dtype: int64

### Как разные цели кредита влияют на его возврат в срок?

In [76]:
def number_purpose(purpose, debt): 
    '''функция, которая сначала создает отфильтрованный
    по параметрe purpose (причина), debt (были ли просрочки) DataFrame, 
    а потом подсчитывает количество строк, удовлетворяющих фильтру '''    
    purpose_list = df[df['purpose_new'] == purpose]  
    purpose_list = purpose_list[purpose_list['debt'] == debt] 
    purpose_list_count = purpose_list['purpose_new'].count()  
    return purpose_list_count

In [77]:
name_columns = ['purpose_count','автомобиль', 'образование', 'свадьба', 'недвижимость', 'ремонт']
request_counts = []
for item_debt in [0, 1]:
    row = []
    row.append(item_debt)
    for item_purpose in ['автомобиль', 'образование', 'свадьба', 'недвижимость', 'ремонт']:
        row.append(number_purpose(item_purpose, item_debt))
    request_counts.append(row)
request_purpose = pd.DataFrame(data = request_counts ,columns = name_columns, index = ['нет просрочки', 'есть просрочка'])
for item in ['автомобиль', 'образование', 'свадьба', 'недвижимость', 'ремонт']:
    request_purpose[item] = request_purpose[item] / request_purpose[item].sum()*100
request_purpose

,purpose_count,автомобиль,образование,свадьба,недвижимость,ремонт
нет просрочки,0,90.587684,90.665994,91.934085,92.53328,94.233937
есть просрочка,1,9.412316,9.334006,8.065915,7.46672,5.766063


Самая большая доля просрочки возникает в случае покупки автомобиля. Самая малая доля просрочки возникает в случае покупки ремонта своего жилья. Не очень далеко ушла доля в случае сделок с недвижимостью, видимо все-таки при покупке жилья в кредит оно является единственным у клиентов, и терять его не очень хочется.

## Общие выводы


Если по итогам вопросов сформировать портрет "положительного клиента", вероятность просрочки которого будет меньше всего, то это будет бездетный(ая) (риск просрочки 7,5%),
женатый(замужняя) (риск просрочки 7,68%), 
с высоким уровнем дохода (4,35%),
который хочет купить недвижимость (7,47% ),
или сделать ремонт (5,76% ).


Самый "неблагоприятный клиент" - это двухдетные (9,44% ), живущие в гражданском браке (9,41% ), или неженатые (9,84% ), с низким уровнем дохода (8,59% ), которые хотят купить автомобиль (9,41%).


В целом, с увеличением детей доля просроченных кредитов будет увеличиваться.

Семейный статус влияет на частоту возникновения просрочек по кредитам, и женатые/вдовые, видимо обладают большей ответственностью, что конечно влияет на отношение к возникшим обязательствам.

При анализе меня смутили значения столбца доход клиента, но я не могу со своей стороны никак на значения повлиять. Тут нужна консультация с заказчиком.
Несмотря на это, прослеживается связь между уровнем дохода и частотой возникновения просрочек, чем выше доход, тем реже просрочки.

Самая большая доля просрочки возникает в случае покупки автомобиля, самая малая доля просрочки возникает в случае покупки и ремонта своего жилья. Скорее всего, при покупке жилья в кредит оно является единственным у клиентов, и терять его не хочется, и поэтому клиенты также относятся внимательнее к возможным просрочкам.